# 使用块的网络

* 与芯片设计中工程师从放置晶体管到逻辑元件再到逻辑块的过程类似
* 神经网络架构的设计也逐渐变得更加抽象
* 研究人员开始从单个神经元的角度思考问题，发展到整个层，现在又转向块，重复层的模式

# 使用块的网络

* 使用块的想法首先出现在牛津大学的视觉几何组（visualgeometry group）的*VGG网络*中

## VGG块

经典卷积神经网络的基本组成部分是下面的这个序列：

1. 带填充以保持分辨率的卷积层
1. 非线性激活函数，如ReLU
1. 池化层，如最大池化层

而一个VGG块与之类似，由一系列卷积层组成，后面再加上用于空间下采样（spatial downsampling）的最大池化层

## VGG块

* $3\times3$卷积核、填充为1（保持高度和宽度）的卷积层
    * 深层且窄的卷积（即$3 \times 3$）比较浅层且宽的卷积更有效
* $2 \times 2$池化窗口、步幅为2（每个块后的分辨率减半）的最大池化层

阅读：使用块的网络\_VGG.pdf

In [2]:
import torch
from d2l import torch as d2l
from torch import nn


def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

## VGG网络

* 原始VGG网络有5个卷积块
    * 其中前两个块各有一个卷积层，后三个块各包含两个卷积层
* 第一个模块有64个输出通道，每个后续模块将输出通道数量翻倍，直到该数字达到512
    * 由于该网络使用8个卷积层和3个全连接层，因此它通常被称为VGG-11

In [3]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

VGG-11

In [4]:
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    # 卷积层部分
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks,
        nn.Flatten(),
        # 全连接层部分
        nn.Linear(out_channels * 7 * 7, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )


net = vgg(conv_arch)

观察每个层输出的形状

In [5]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, "output shape:\t", X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


## Classification

[Classification](https://cv.gluon.ai/model_zoo/classification.html)